In [1]:
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
train_dir = '/home/ammar/Desktop/U/semester-8/fyp/data-rescaled-split/train'
test_dir = '/home/ammar/Desktop/U/semester-8/fyp/data-rescaled-split/test'
val_dir = '/home/ammar/Desktop/U/semester-8/fyp/data-rescaled-split/val'
visualization_dir = '/home/ammar/Desktop/U/semester-8/fyp/data-rescaled-split/augmented'  # For storing augmented images generated in a directory
log_dir = '/home/ammar/Desktop/U/semester-8/fyp/data-rescaled-split/logs'  # For stroing tensorboard logs

image_height = 224
image_width  = 224
batch_size = 32
epochs = 10

In [5]:
def build_model():
    # constructing the model
    model = applications.MobileNetV3Large(weights="imagenet", include_top=False, input_shape=(image_width, image_height, 3),
                                  pooling='avg')

    # We don't want 10 layers from last, practice of transfer learning
    for layer in model.layers[:-10]:
        layer.trainable = False

    # Adding custom Layers
    x = model.output
    
    predictions = Dense(15, activation="softmax")(x)   # Connecting our required output layer with the MobileNet

    # creating the final model
    model_final = Model(inputs=model.input, outputs=predictions)   
    
    return model_final

model_final = build_model()

# compile the model
model_final.compile(loss="categorical_crossentropy",
                    optimizer=optimizers.Adam(learning_rate=0.001),  # Adam gives better results on this learning rate
                    metrics=["accuracy", "Precision"])

train_datagen = ImageDataGenerator(
    fill_mode="nearest",   # If any pixels are empty should fill using this method
    zoom_range=0.2,        # Number greater than 0.2 would give unwanted augmented images
    width_shift_range = 0.2,    # Number greater than 0.2 would give unwanted augmented images
    height_shift_range = 0.2,    # Number greater than 0.2 would give unwanted augmented images
    shear_range = 0.2,          # Number greater than 0.2 would give unwanted augmented images
    rotation_range=180
)


validation_datagen = ImageDataGenerator(
    
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    save_to_dir = visualization_dir,
    target_size=(image_height, image_width),  
    batch_size=batch_size,
    class_mode="categorical")  # As we want multuple categories

validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    target_size=(image_height, image_width),
    class_mode="categorical")


tensorboard = TensorBoard(log_dir=log_dir)  # Tensorboard callback used for plotting

# Train the model
model_final.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[tensorboard])

Found 3746 images belonging to 15 classes.
Found 410 images belonging to 15 classes.
Epoch 1/10
118/118 [==============================] - 256s 2s/step - loss: 0.9445 - accuracy: 0.7274 - precision: 0.9477 - val_loss: 0.7954 - val_accuracy: 0.7659 - val_precision: 0.8125
Epoch 2/10
118/118 [==============================] - 213s 2s/step - loss: 0.1817 - accuracy: 0.9562 - precision: 0.9761 - val_loss: 0.5378 - val_accuracy: 0.8220 - val_precision: 0.8508
Epoch 3/10
118/118 [==============================] - 209s 2s/step - loss: 0.1142 - accuracy: 0.9701 - precision: 0.9819 - val_loss: 0.3539 - val_accuracy: 0.8951 - val_precision: 0.9191
Epoch 4/10
118/118 [==============================] - 192s 2s/step - loss: 0.0835 - accuracy: 0.9784 - precision: 0.9840 - val_loss: 0.1725 - val_accuracy: 0.9415 - val_precision: 0.9547
Epoch 5/10
118/118 [==============================] - 187s 2s/step - loss: 0.0725 - accuracy: 0.9789 - precision: 0.9846 - val_loss: 0.2196 - val_accuracy: 0.9341 - va

In [8]:
# Now evaluating our model on test data
test_datagen = ImageDataGenerator(
    fill_mode="nearest",
    zoom_range=0.3,
    rotation_range=30
)

test_generator = test_datagen.flow_from_directory(
                    test_dir,
                    target_size=(image_height, image_width),
                    class_mode="categorical")

Found 455 images belonging to 15 classes.


In [13]:
model_final.evaluate(test_generator)

15/15 [==============================] - 13s 862ms/step - loss: 0.1388 - accuracy: 0.9495 - precision: 0.9514


[0.1388421654701233, 0.9494505524635315, 0.951434850692749]

In [6]:
model_final.save('fifth-data-augmented-background-model.hd5')

INFO:tensorflow:Assets written to: fifth-data-augmented-background-model.hd5/assets


In [7]:
train_generator.class_indices 

{'1000_back': 0,
 '1000_front': 1,
 '100_back': 2,
 '100_front': 3,
 '10_back': 4,
 '10_front': 5,
 '20_back': 6,
 '20_front': 7,
 '5000_back': 8,
 '5000_front': 9,
 '500_back': 10,
 '500_front': 11,
 '50_back': 12,
 '50_front': 13,
 'background': 14}